# Imports & Reading Data

In [1]:
import os
import ipdb
import argparse
import numpy as np
import pandas as pd
import numpy.random as random
from sklearn.inspection import permutation_importance
import networkx as nx
from progressbar import ProgressBar
import matplotlib.pyplot as plt
import gzip
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  roc_auc_score, roc_curve
import csv
from joblib import dump, load
import json
pd.options.display.max_columns = 9999999

# Learning Technique #2: Classical + Underasmpling in training only

## For $r$ = 0.7
Before undersampling, take 30% for testing then the 70% take it for training and do undersampling on it

### Undersample the .json file for training

In [3]:
a = input("Restart Jupyter Notebook and Run Import Cell Then Cell Just After This Cell")
print(a)

Restart Jupyter Notebook and Run Import Cell Then Cell Just After This Cell



In [2]:
# Open the undersampled.txt file and read the relevant lines
with open('train_df_no_features_undersampled_r0.7.txt', 'r') as f:
    reader = csv.reader(f, delimiter=' ')
    relevant_tuples = {(int(row[0]), str(row[1]), str(row[2])) for i, row in enumerate(reader) if i > 0} | {(int(row[0]), int(row[2]), int(row[1])) for i, row in enumerate(reader) if i > 0}

In [3]:
relevant_tuples

{(1554772132, '150.57.182.64', '155.153.42.188'),
 (1554776654, '104.243.199.192', '163.33.140.115'),
 (1554792441, '182.247.80.65', '60.53.92.64'),
 (1554774280, '211.128.27.160', '36.127.79.56'),
 (1554758305, '117.79.113.71', '131.142.28.32'),
 (1554747569, '104.15.196.220', '150.57.77.103'),
 (1554766613, '133.63.69.124', '49.232.19.186'),
 (1554745303, '183.33.90.27', '82.204.166.93'),
 (1554780690, '222.227.227.32', '223.207.55.73'),
 (1554788013, '202.204.60.245', '216.68.133.206'),
 (1554759600, '193.60.159.37', '202.0.26.220'),
 (1554749457, '133.106.13.240', '192.219.107.24'),
 (1554786173, '133.237.63.172', '46.234.159.217'),
 (1554793734, '133.27.39.237', '61.38.66.2'),
 (1554739925, '111.27.132.154', '112.112.220.34'),
 (1554775688, '139.43.132.146', '150.57.248.121'),
 (1554777074, '131.142.17.103', '172.249.28.99'),
 (1554736157, '182.59.60.16', '223.198.153.51'),
 (1554738693, '150.57.93.108', '216.68.133.234'),
 (1554767424, '104.120.159.251', '163.33.238.178'),
 (1554

In [4]:
len(relevant_tuples)

15473766

In [5]:
c = 0
with gzip.open('/Mawi2023/mawi_5_injected_H15000.gz', 'rt') as in_f, open('filtered_mawi_5_injected_H15000_r0.7.json', 'w') as out_f:
    for line in in_f:
        data = json.loads(line)
        tup1 = (int(data['t']), str(data['u']), str(data['v']))
        if tup1 in relevant_tuples:
            encoded_data = json.dumps(data)
            out_f.write(encoded_data)
            out_f.write('\n')
            c += 1
            if c % 1000000 == 0:  # Check if c is a multiple of 1,000,000
                print("Processed", c, "steps")

Processed 1000000 steps
Processed 2000000 steps
Processed 3000000 steps
Processed 4000000 steps
Processed 5000000 steps
Processed 6000000 steps
Processed 7000000 steps
Processed 8000000 steps
Processed 9000000 steps
Processed 10000000 steps
Processed 11000000 steps
Processed 12000000 steps
Processed 13000000 steps
Processed 14000000 steps
Processed 15000000 steps


In [6]:
with open('filtered_mawi_5_injected_H15000_r0.7.json', 'r') as file:
    row_count = sum(1 for row in file)
print("The number of rows in the file is:", row_count)

The number of rows in the file is: 15473766


In [7]:
15473766 == row_count

True

In [8]:
fraud_count = 0
normal_count = 0

# Open the JSON file and iterate through each line
with open('filtered_mawi_5_injected_H15000_r0.7.json', 'r') as file:
    for line in file:
        # Load each line as a JSON object
        data = json.loads(line)
        # Check if the value of 'is_fraud' is equal to 1
        if data['is_fraud'] == 1:
            fraud_count += 1
        else:
            normal_count += 1

print("Total is_fraud = 1:", fraud_count)
print("Total is_fraud = 0:", normal_count)

Total is_fraud = 1: 7736883
Total is_fraud = 0: 7736883


In [9]:
a = input("Restart Jupyter Notebook and Run Import Cell Then Cell Just After This Cell")
print(a)

Restart Jupyter Notebook and Run Import Cell Then Cell Just After This Cell



### Read in the `filtered...`

In [3]:
df_filtered = pd.read_json('filtered_mawi_5_injected_H15000_r0.7.json', lines=True)

In [4]:
df_filtered

,u,t,bip,cost,history_type,is_fraud,v,history_size,v_same_degree,weighted_degrees_min,degrees_min,link_weights_median,v_greater_weighted_degree,u_same_degree,number_of_nodes,u_greater_degree,u_v_same_weight,number_of_links,u_v_weight,link_weights_nb_two,v_weighted_degree,v_greater_degree,link_weights_max,link_weights_nb_one,total_weight,link_weights_min,degrees_nb_two,u_v_greater_weight,v_same_weighted_degree,degrees_nb_one,weighted_degrees_median,u_same_weighted_degree,v_degree,weighted_degrees_max,u_degree,degrees_median,u_weighted_degree,degrees_max,weighted_degrees_nb_two,u_greater_weighted_degree,weighted_degrees_nb_one
0,101.20.243.169,1554735600,False,0.000035,H,0,203.77.115.33,15000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101.73.213.30,1554735600,False,0.000036,H,1,133.27.105.85,15000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.20.112.101,1554735600,False,0.000047,H,0,202.0.26.220,15000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103.108.158.231,1554735600,False,0.000036,H,0,203.77.116.213,15000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,103.192.235.218,1554735600,False,0.000035,H,0,202.198.30.236,15000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15473761,103.18.48.234,1554795472,False,0.000051,H,1,49.235.198.148,15000,6471.0,1.0,1.0,1.0,3379.0,6471.0,8517.0,2046.0,7895.0,10487.0,1.0,1366.0,1.0,2046.0,5.0,7895.0,15000.0,1.0,856.0,2592.0,5138.0,6471.0,1.0,5138.0,1.0,850.0,1.0,1.0,1.0,842.0,1149.0,3379.0,5138.0
15473762,103.209.46.221,1554795472,False,0.000083,H,1,42.189.104.165,15000,6471.0,1.0,1.0,1.0,3380.0,6471.0,8517.0,2046.0,7895.0,10489.0,1.0,1369.0,1.0,2046.0,5.0,7895.0,15000.0,1.0,854.0,2594.0,5137.0,6471.0,1.0,5137.0,1.0,850.0,1.0,1.0,1.0,842.0,1150.0,3380.0,5137.0
15473763,103.222.193.242,1554795472,False,0.000059,H,0,133.149.254.43,15000,6473.0,1.0,1.0,1.0,3378.0,6473.0,8522.0,2049.0,7906.0,10496.0,1.0,1364.0,1.0,2049.0,5.0,7906.0,15000.0,1.0,856.0,2590.0,5144.0,6473.0,1.0,5144.0,1.0,850.0,1.0,1.0,1.0,842.0,1148.0,3378.0,5144.0
15473764,103.236.150.189,1554795472,False,0.000061,H,1,111.211.230.68,15000,6476.0,1.0,1.0,1.0,3377.0,6476.0,8524.0,2048.0,7906.0,10496.0,1.0,1364.0,1.0,2048.0,5.0,7906.0,15000.0,1.0,855.0,2590.0,5147.0,6476.0,1.0,5147.0,1.0,850.0,1.0,1.0,1.0,842.0,1147.0,3377.0,5147.0


In [5]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15473766 entries, 0 to 15473765
Data columns (total 41 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   u                          object 
 1   t                          int64  
 2   bip                        bool   
 3   cost                       float64
 4   history_type               object 
 5   is_fraud                   int64  
 6   v                          object 
 7   history_size               int64  
 8   v_same_degree              float64
 9   weighted_degrees_min       float64
 10  degrees_min                float64
 11  link_weights_median        float64
 12  v_greater_weighted_degree  float64
 13  u_same_degree              float64
 14  number_of_nodes            float64
 15  u_greater_degree           float64
 16  u_v_same_weight            float64
 17  number_of_links            float64
 18  u_v_weight                 float64
 19  link_weights_nb_two        float64
 20  

In [6]:
df_filtered.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15473766 entries, 0 to 15473765
Columns: 41 entries, u to weighted_degrees_nb_one
dtypes: bool(1), float64(34), int64(3), object(3)
memory usage: 7.1 GB


In [7]:
df_filtered = df_filtered.dropna()
print("Updated DataFrame without NaN:")
df_filtered

Updated DataFrame without NaN:


,u,t,bip,cost,history_type,is_fraud,v,history_size,v_same_degree,weighted_degrees_min,degrees_min,link_weights_median,v_greater_weighted_degree,u_same_degree,number_of_nodes,u_greater_degree,u_v_same_weight,number_of_links,u_v_weight,link_weights_nb_two,v_weighted_degree,v_greater_degree,link_weights_max,link_weights_nb_one,total_weight,link_weights_min,degrees_nb_two,u_v_greater_weight,v_same_weighted_degree,degrees_nb_one,weighted_degrees_median,u_same_weighted_degree,v_degree,weighted_degrees_max,u_degree,degrees_median,u_weighted_degree,degrees_max,weighted_degrees_nb_two,u_greater_weighted_degree,weighted_degrees_nb_one
1323,200.135.140.210,1554735604,False,0.000048,H,1,203.77.46.114,15000,8176.0,1.0,1.0,1.0,3331.0,8176.0,9989.0,1813.0,8414.0,10664.0,1.0,1155.0,1.0,1813.0,5.0,8414.0,15000.0,1.0,902.0,2250.0,6658.0,8176.0,1.0,6658.0,1.0,1730.0,1.0,1.0,1.0,1371.0,1259.0,3331.0,6658.0
1324,202.0.26.220,1554735604,False,0.000046,H,0,202.31.77.76,15000,1.0,1.0,1.0,1.0,0.0,8179.0,9992.0,1813.0,8416.0,10667.0,1.0,1156.0,1737.0,0.0,5.0,8416.0,15000.0,1.0,902.0,2251.0,1.0,8179.0,1.0,6661.0,1374.0,1737.0,1.0,1.0,1.0,1374.0,1258.0,3331.0,6661.0
1325,202.0.26.220,1554735604,False,0.000058,H,0,221.198.188.13,15000,1.0,1.0,1.0,1.0,0.0,8183.0,9994.0,1811.0,361.0,10668.0,5.0,1150.0,1781.0,0.0,5.0,8418.0,15000.0,1.0,904.0,0.0,1.0,8183.0,1.0,448.0,1392.0,1781.0,1.0,1.0,5.0,1392.0,1251.0,611.0,6667.0
1326,202.0.30.234,1554735604,False,0.000064,H,0,213.44.2.215,15000,1.0,1.0,1.0,1.0,5.0,8192.0,10003.0,1811.0,265.0,10672.0,4.0,1147.0,471.0,4.0,5.0,8421.0,15000.0,1.0,906.0,354.0,1.0,8192.0,1.0,390.0,311.0,1794.0,1.0,1.0,4.0,1401.0,1256.0,1054.0,6669.0
1327,202.0.30.234,1554735604,False,0.000065,H,0,213.44.71.24,15000,1.0,1.0,1.0,1.0,5.0,905.0,10000.0,905.0,354.0,10669.0,5.0,1145.0,477.0,4.0,5.0,8418.0,15000.0,1.0,905.0,0.0,1.0,8190.0,1.0,118.0,311.0,1794.0,2.0,1.0,6.0,1401.0,1254.0,493.0,6667.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15473761,103.18.48.234,1554795472,False,0.000051,H,1,49.235.198.148,15000,6471.0,1.0,1.0,1.0,3379.0,6471.0,8517.0,2046.0,7895.0,10487.0,1.0,1366.0,1.0,2046.0,5.0,7895.0,15000.0,1.0,856.0,2592.0,5138.0,6471.0,1.0,5138.0,1.0,850.0,1.0,1.0,1.0,842.0,1149.0,3379.0,5138.0
15473762,103.209.46.221,1554795472,False,0.000083,H,1,42.189.104.165,15000,6471.0,1.0,1.0,1.0,3380.0,6471.0,8517.0,2046.0,7895.0,10489.0,1.0,1369.0,1.0,2046.0,5.0,7895.0,15000.0,1.0,854.0,2594.0,5137.0,6471.0,1.0,5137.0,1.0,850.0,1.0,1.0,1.0,842.0,1150.0,3380.0,5137.0
15473763,103.222.193.242,1554795472,False,0.000059,H,0,133.149.254.43,15000,6473.0,1.0,1.0,1.0,3378.0,6473.0,8522.0,2049.0,7906.0,10496.0,1.0,1364.0,1.0,2049.0,5.0,7906.0,15000.0,1.0,856.0,2590.0,5144.0,6473.0,1.0,5144.0,1.0,850.0,1.0,1.0,1.0,842.0,1148.0,3378.0,5144.0
15473764,103.236.150.189,1554795472,False,0.000061,H,1,111.211.230.68,15000,6476.0,1.0,1.0,1.0,3377.0,6476.0,8524.0,2048.0,7906.0,10496.0,1.0,1364.0,1.0,2048.0,5.0,7906.0,15000.0,1.0,855.0,2590.0,5147.0,6476.0,1.0,5147.0,1.0,850.0,1.0,1.0,1.0,842.0,1147.0,3377.0,5147.0


In [8]:
df_filtered = df_filtered.drop(["t", "u", "v", "history_type", "history_size", "bip", "cost"], axis="columns")
df_filtered

,is_fraud,v_same_degree,weighted_degrees_min,degrees_min,link_weights_median,v_greater_weighted_degree,u_same_degree,number_of_nodes,u_greater_degree,u_v_same_weight,number_of_links,u_v_weight,link_weights_nb_two,v_weighted_degree,v_greater_degree,link_weights_max,link_weights_nb_one,total_weight,link_weights_min,degrees_nb_two,u_v_greater_weight,v_same_weighted_degree,degrees_nb_one,weighted_degrees_median,u_same_weighted_degree,v_degree,weighted_degrees_max,u_degree,degrees_median,u_weighted_degree,degrees_max,weighted_degrees_nb_two,u_greater_weighted_degree,weighted_degrees_nb_one
1323,1,8176.0,1.0,1.0,1.0,3331.0,8176.0,9989.0,1813.0,8414.0,10664.0,1.0,1155.0,1.0,1813.0,5.0,8414.0,15000.0,1.0,902.0,2250.0,6658.0,8176.0,1.0,6658.0,1.0,1730.0,1.0,1.0,1.0,1371.0,1259.0,3331.0,6658.0
1324,0,1.0,1.0,1.0,1.0,0.0,8179.0,9992.0,1813.0,8416.0,10667.0,1.0,1156.0,1737.0,0.0,5.0,8416.0,15000.0,1.0,902.0,2251.0,1.0,8179.0,1.0,6661.0,1374.0,1737.0,1.0,1.0,1.0,1374.0,1258.0,3331.0,6661.0
1325,0,1.0,1.0,1.0,1.0,0.0,8183.0,9994.0,1811.0,361.0,10668.0,5.0,1150.0,1781.0,0.0,5.0,8418.0,15000.0,1.0,904.0,0.0,1.0,8183.0,1.0,448.0,1392.0,1781.0,1.0,1.0,5.0,1392.0,1251.0,611.0,6667.0
1326,0,1.0,1.0,1.0,1.0,5.0,8192.0,10003.0,1811.0,265.0,10672.0,4.0,1147.0,471.0,4.0,5.0,8421.0,15000.0,1.0,906.0,354.0,1.0,8192.0,1.0,390.0,311.0,1794.0,1.0,1.0,4.0,1401.0,1256.0,1054.0,6669.0
1327,0,1.0,1.0,1.0,1.0,5.0,905.0,10000.0,905.0,354.0,10669.0,5.0,1145.0,477.0,4.0,5.0,8418.0,15000.0,1.0,905.0,0.0,1.0,8190.0,1.0,118.0,311.0,1794.0,2.0,1.0,6.0,1401.0,1254.0,493.0,6667.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15473761,1,6471.0,1.0,1.0,1.0,3379.0,6471.0,8517.0,2046.0,7895.0,10487.0,1.0,1366.0,1.0,2046.0,5.0,7895.0,15000.0,1.0,856.0,2592.0,5138.0,6471.0,1.0,5138.0,1.0,850.0,1.0,1.0,1.0,842.0,1149.0,3379.0,5138.0
15473762,1,6471.0,1.0,1.0,1.0,3380.0,6471.0,8517.0,2046.0,7895.0,10489.0,1.0,1369.0,1.0,2046.0,5.0,7895.0,15000.0,1.0,854.0,2594.0,5137.0,6471.0,1.0,5137.0,1.0,850.0,1.0,1.0,1.0,842.0,1150.0,3380.0,5137.0
15473763,0,6473.0,1.0,1.0,1.0,3378.0,6473.0,8522.0,2049.0,7906.0,10496.0,1.0,1364.0,1.0,2049.0,5.0,7906.0,15000.0,1.0,856.0,2590.0,5144.0,6473.0,1.0,5144.0,1.0,850.0,1.0,1.0,1.0,842.0,1148.0,3378.0,5144.0
15473764,1,6476.0,1.0,1.0,1.0,3377.0,6476.0,8524.0,2048.0,7906.0,10496.0,1.0,1364.0,1.0,2048.0,5.0,7906.0,15000.0,1.0,855.0,2590.0,5147.0,6476.0,1.0,5147.0,1.0,850.0,1.0,1.0,1.0,842.0,1147.0,3377.0,5147.0


In [9]:
# If they are not equal it is ok now because we removed the NaN
df_filtered["is_fraud"].value_counts()

1    7736267
0    7736176
Name: is_fraud, dtype: int64

### Train

In [10]:
# Specify the features (X) and the target variable (y)
X_train = df_filtered.drop('is_fraud', axis=1)
y_train = df_filtered['is_fraud']

In [11]:
X_train

,v_same_degree,weighted_degrees_min,degrees_min,link_weights_median,v_greater_weighted_degree,u_same_degree,number_of_nodes,u_greater_degree,u_v_same_weight,number_of_links,u_v_weight,link_weights_nb_two,v_weighted_degree,v_greater_degree,link_weights_max,link_weights_nb_one,total_weight,link_weights_min,degrees_nb_two,u_v_greater_weight,v_same_weighted_degree,degrees_nb_one,weighted_degrees_median,u_same_weighted_degree,v_degree,weighted_degrees_max,u_degree,degrees_median,u_weighted_degree,degrees_max,weighted_degrees_nb_two,u_greater_weighted_degree,weighted_degrees_nb_one
1323,8176.0,1.0,1.0,1.0,3331.0,8176.0,9989.0,1813.0,8414.0,10664.0,1.0,1155.0,1.0,1813.0,5.0,8414.0,15000.0,1.0,902.0,2250.0,6658.0,8176.0,1.0,6658.0,1.0,1730.0,1.0,1.0,1.0,1371.0,1259.0,3331.0,6658.0
1324,1.0,1.0,1.0,1.0,0.0,8179.0,9992.0,1813.0,8416.0,10667.0,1.0,1156.0,1737.0,0.0,5.0,8416.0,15000.0,1.0,902.0,2251.0,1.0,8179.0,1.0,6661.0,1374.0,1737.0,1.0,1.0,1.0,1374.0,1258.0,3331.0,6661.0
1325,1.0,1.0,1.0,1.0,0.0,8183.0,9994.0,1811.0,361.0,10668.0,5.0,1150.0,1781.0,0.0,5.0,8418.0,15000.0,1.0,904.0,0.0,1.0,8183.0,1.0,448.0,1392.0,1781.0,1.0,1.0,5.0,1392.0,1251.0,611.0,6667.0
1326,1.0,1.0,1.0,1.0,5.0,8192.0,10003.0,1811.0,265.0,10672.0,4.0,1147.0,471.0,4.0,5.0,8421.0,15000.0,1.0,906.0,354.0,1.0,8192.0,1.0,390.0,311.0,1794.0,1.0,1.0,4.0,1401.0,1256.0,1054.0,6669.0
1327,1.0,1.0,1.0,1.0,5.0,905.0,10000.0,905.0,354.0,10669.0,5.0,1145.0,477.0,4.0,5.0,8418.0,15000.0,1.0,905.0,0.0,1.0,8190.0,1.0,118.0,311.0,1794.0,2.0,1.0,6.0,1401.0,1254.0,493.0,6667.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15473761,6471.0,1.0,1.0,1.0,3379.0,6471.0,8517.0,2046.0,7895.0,10487.0,1.0,1366.0,1.0,2046.0,5.0,7895.0,15000.0,1.0,856.0,2592.0,5138.0,6471.0,1.0,5138.0,1.0,850.0,1.0,1.0,1.0,842.0,1149.0,3379.0,5138.0
15473762,6471.0,1.0,1.0,1.0,3380.0,6471.0,8517.0,2046.0,7895.0,10489.0,1.0,1369.0,1.0,2046.0,5.0,7895.0,15000.0,1.0,854.0,2594.0,5137.0,6471.0,1.0,5137.0,1.0,850.0,1.0,1.0,1.0,842.0,1150.0,3380.0,5137.0
15473763,6473.0,1.0,1.0,1.0,3378.0,6473.0,8522.0,2049.0,7906.0,10496.0,1.0,1364.0,1.0,2049.0,5.0,7906.0,15000.0,1.0,856.0,2590.0,5144.0,6473.0,1.0,5144.0,1.0,850.0,1.0,1.0,1.0,842.0,1148.0,3378.0,5144.0
15473764,6476.0,1.0,1.0,1.0,3377.0,6476.0,8524.0,2048.0,7906.0,10496.0,1.0,1364.0,1.0,2048.0,5.0,7906.0,15000.0,1.0,855.0,2590.0,5147.0,6476.0,1.0,5147.0,1.0,850.0,1.0,1.0,1.0,842.0,1147.0,3377.0,5147.0


In [12]:
y_train

1323        1
1324        0
1325        0
1326        0
1327        0
           ..
15473761    1
15473762    1
15473763    0
15473764    1
15473765    1
Name: is_fraud, Length: 15472443, dtype: int64

In [13]:
rf_classifier = RandomForestClassifier(random_state=42, n_jobs = -1)

rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [14]:
dump(rf_classifier, 'Mawi_5Perc_H15000_r0.7.joblib') 

['Mawi_5Perc_H15000_r0.7.joblib']

In [15]:
a = input("Restart Jupyter Notebook and Run Import Cell Then Cell Just After This Cell")
print(a)

Restart Jupyter Notebook and Run Import Cell Then Cell Just After This Cell



### Preprocess Testing Data
- Get the links based on `test_df_no_features` and all their respective features
- Remove NaN
- Shuffle
- Remove the unused columns such as t, u, v...
- Divide between X and y variables

In [2]:
with open('test_df_no_features_r0.7.txt', 'r') as f:
    reader = csv.reader(f, delimiter=' ')
    relevant_tuples = {(int(row[0]), str(row[1]), str(row[2])) for i, row in enumerate(reader) if i > 0} | {(int(row[0]), int(row[2]), int(row[1])) for i, row in enumerate(reader) if i > 0}

In [3]:
relevant_tuples

{(1554805807, '203.77.118.202', '223.232.142.168'),
 (1554803905, '133.12.69.62', '57.56.149.37'),
 (1554810343, '139.180.247.69', '150.57.250.126'),
 (1554804257, '131.142.187.7', '172.249.25.177'),
 (1554807001, '111.37.73.26', '150.57.248.121'),
 (1554805425, '150.57.248.121', '27.142.70.68'),
 (1554809641, '113.149.117.140', '203.77.125.43'),
 (1554805636, '202.22.168.33', '210.229.226.108'),
 (1554813201, '120.180.155.149', '202.198.31.184'),
 (1554819289, '106.132.123.46', '133.27.10.155'),
 (1554819795, '203.178.86.227', '80.213.53.141'),
 (1554795732, '163.33.44.18', '64.22.165.172'),
 (1554818110, '142.249.15.54', '202.0.26.220'),
 (1554820027, '203.77.113.192', '74.193.202.178'),
 (1554796289, '111.38.39.179', '112.107.4.6'),
 (1554795995, '131.142.188.190', '172.249.28.78'),
 (1554798303, '163.33.107.203', '17.199.46.206'),
 (1554815424, '150.57.183.2', '172.249.30.58'),
 (1554799170, '163.33.166.44', '31.113.147.98'),
 (1554820507, '14.133.116.57', '202.0.26.220'),
 (155479

In [4]:
len(relevant_tuples)

70318996

In [5]:
a = input("Provide the known total number of rows based on df_no_features")
print(a)


Provide the known total number of rows based on df_no_features



In [5]:
total_rows = 234396654  # Provide the known total number of rows based on df_no_features

percentage_to_skip = 0.7
rows_to_skip = int(total_rows * percentage_to_skip)

c = 0
with gzip.open('/Mawi2023/mawi_5_injected_H15000.gz', 'rt') as in_f, gzip.open('to_test_mawi_5_injected_H15000_r0.7.json.gz', 'wt') as out_f:
    for i, line in enumerate(in_f):
        if i < rows_to_skip:
            continue
        data = json.loads(line)
        tup1 = (int(data['t']), str(data['u']), str(data['v']))
        if tup1 in relevant_tuples:
            encoded_data = json.dumps(data)
            out_f.write(encoded_data)
            out_f.write('\n')
            c += 1
            if c % 1000000 == 0: 
                print("Processed", c, "steps")

Processed 1000000 steps
Processed 2000000 steps
Processed 3000000 steps
Processed 4000000 steps
Processed 5000000 steps
Processed 6000000 steps
Processed 7000000 steps
Processed 8000000 steps
Processed 9000000 steps
Processed 10000000 steps
Processed 11000000 steps
Processed 12000000 steps
Processed 13000000 steps
Processed 14000000 steps
Processed 15000000 steps
Processed 16000000 steps
Processed 17000000 steps
Processed 18000000 steps
Processed 19000000 steps
Processed 20000000 steps
Processed 21000000 steps
Processed 22000000 steps
Processed 23000000 steps
Processed 24000000 steps
Processed 25000000 steps
Processed 26000000 steps
Processed 27000000 steps
Processed 28000000 steps
Processed 29000000 steps
Processed 30000000 steps
Processed 31000000 steps
Processed 32000000 steps
Processed 33000000 steps
Processed 34000000 steps
Processed 35000000 steps
Processed 36000000 steps
Processed 37000000 steps
Processed 38000000 steps
Processed 39000000 steps
Processed 40000000 steps
Processed

In [6]:
with gzip.open('to_test_mawi_5_injected_H15000_r0.7.json.gz', 'r') as file:
    row_count = sum(1 for row in file)
print("The number of rows in the file is:", row_count)

The number of rows in the file is: 70318996


In [7]:
1+1

2

In [8]:
# Initialize counter
fraud_count = 0
normal_count = 0

# Open the JSON file and iterate through each line
with gzip.open('to_test_mawi_5_injected_H15000_r0.7.json.gz', 'rt') as file:
    for line in file:
        # Load each line as a JSON object
        data = json.loads(line)
        # Check if the value of 'is_fraud' is equal to 1
        if data['is_fraud'] == 1:
            fraud_count += 1
        else:
            normal_count += 1

print("Total is_fraud = 1:", fraud_count)
print("Total is_fraud = 0:", normal_count)

Total is_fraud = 1: 3424862
Total is_fraud = 0: 66894134


### Test

In [9]:
a = input("Restart Jupyter Notebook and Run Import Cell Then Cell Just After This Cell")
print(a)

Restart Jupyter Notebook and Run Import Cell Then Cell Just After This Cell



In [2]:
# Load the trained classifier
clf = load('Mawi_5Perc_H15000_r0.7.joblib')

In [3]:
# Read the gzipped JSON file into a DataFrame in chunks
chunk_size = 1000000
roc_auc_scores = []

chunk_counter = 0

# Open the gzipped JSON file
with gzip.open("to_test_mawi_5_injected_H15000_r0.7.json.gz", "rb") as f:
    for chunk in pd.read_json(f, lines=True, chunksize=chunk_size):
        # Drop NaN values and unnecessary columns
        chunk = chunk.dropna()
        chunk = chunk.drop(["t", "u", "v", "history_type", "history_size", "bip", "cost"], axis="columns")
        
        # Specify features (X) and target variable (y)
        X_chunk = chunk.drop('is_fraud', axis=1)
        y_chunk = chunk['is_fraud']
        
        # Predict labels for the chunk
        y_pred_chunk = clf.predict(X_chunk)
        roc_auc_chunk = roc_auc_score(y_chunk, clf.predict_proba(X_chunk)[:, 1])
        roc_auc_scores.append(roc_auc_chunk)
        chunk_counter += 1
        print("Iteration:", chunk_counter)
        print("roc_auc_chunk:", round(roc_auc_chunk, 4))
        print("--------------------------------------------------------------")

        
avg_roc_auc = sum(roc_auc_scores) / len(roc_auc_scores)


print(f"Average ROC-AUC Score: {avg_roc_auc:.4f}")

Iteration: 1
roc_auc_chunk: 0.9884
--------------------------------------------------------------
Iteration: 2
roc_auc_chunk: 0.9893
--------------------------------------------------------------
Iteration: 3
roc_auc_chunk: 0.99
--------------------------------------------------------------
Iteration: 4
roc_auc_chunk: 0.9901
--------------------------------------------------------------
Iteration: 5
roc_auc_chunk: 0.9897
--------------------------------------------------------------
Iteration: 6
roc_auc_chunk: 0.9891
--------------------------------------------------------------
Iteration: 7
roc_auc_chunk: 0.9893
--------------------------------------------------------------
Iteration: 8
roc_auc_chunk: 0.9897
--------------------------------------------------------------
Iteration: 9
roc_auc_chunk: 0.989
--------------------------------------------------------------
Iteration: 10
roc_auc_chunk: 0.9886
--------------------------------------------------------------
Iteration: 11
roc_auc_